# Detecting Presence
****

### Background Subtraction

A technique called background subtraction makes it possible to detect the presence of people or other objects in a scene, and to distinguish the pixels which belong to them from those which do not. The technique operates by comparing each frame of video with a stored image of the scene's background, captured at a point in time when the scene was known to be empty. For every pixel in the frame, the absolute difference is computed between its color and that of its corresponding pixel in the stored background image; areas which are very different from the background are likely to represent objects of interest. Background subtraction works well in heterogeneous environments, but it is very sensitive to changes in lighting conditions, and depends on objects of interest having sufficient contrast against the background scene.

The advantage of Background Subtraction is it's ability to track objects that have entered the scene but have stopped moving.  For example, a car that has pulled over and parked on the side of the road.  

### How do you create a background frame?

In most situations you will not have access to a clean background frame.  We will need a process to create one for a dynamic scene.  To create a background frame we are going to capture 100 frames of the scene with the camera.  Then average the frames together to create a background frame.

In [1]:
import cv2
import numpy as np

# Initialize the webcam feed
cap = cv2.VideoCapture(0)

# Check if webcam is successfully opened
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Parameters
num_frames_for_background = 100
count = 0

# Initialize a running sum of frames
background_accumulator = None

while count < num_frames_for_background:
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame.")
        break

    # Convert the frame to float
    float_frame = frame.astype(np.float32)

    # Initialize or accumulate the frames
    if background_accumulator is None:
        background_accumulator = float_frame
    else:
        background_accumulator += float_frame

    count += 1

    cv2.imshow("Capturing Background", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Calculate the average to obtain the background frame
background_frame = (background_accumulator / num_frames_for_background).astype(np.uint8)

# Show the background frame
cv2.imshow("Background Frame", background_frame)
cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()



: 

### Background Subtraction Pipeline

<ol>
    <li>Background Model Creation: A reference frame or a set of frames, usually when no objects of interest are present, is chosen as the background model.</li>
    <li>Difference Computation: For every incoming frame, compute the absolute difference between its pixel values and the corresponding pixel values of the background model.</li>
    <li>Thresholding: Pixels where the difference exceeds a certain threshold are marked as foreground pixels, and others as background.</li>
    <li>Post-processing: Apply techniques like morphological operations to remove noise and fill small gaps.</li>
</ol>


In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

ret, background_img = cap.read()
alpha = 0.02  # Weight factor to update the background. Smaller values make the update slower.

while ret:
    ret, current_frame_img = cap.read()

    if not ret:
        break

    difference = cv2.absdiff(background_img, current_frame_img)
    grayscale = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)

    _, thresholded = cv2.threshold(grayscale, 30, 255, cv2.THRESH_BINARY)

    # Noise reduction using morphological operations
    kernel = np.ones((5, 5), np.uint8)
    opened = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)

    # Find contours and draw bounding boxes
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter out small areas to reduce noise
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(current_frame_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Update the background model
    background_img = cv2.addWeighted(background_img, 1 - alpha, current_frame_img, alpha, 0)

    cv2.imshow('Presence Detection', thresholded)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


: 

### Combine Frame Differencing and Background Subtraction

In [1]:
import cv2

# Initialize the webcam feed
cap = cv2.VideoCapture(0)

# Check if webcam is successfully opened
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Get the first frame for initialization
ret, prev_frame = cap.read()
ret, current_frame = cap.read()

# Initialize the background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

while ret:
    # Frame Differencing
    frame_diff = cv2.absdiff(prev_frame, current_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, thresholded_diff = cv2.threshold(gray_diff, 25, 255, cv2.THRESH_BINARY)

    # Background Subtraction
    fg_mask = bg_subtractor.apply(current_frame)

    # Combine the results of both methods
    combined_mask = cv2.bitwise_or(thresholded_diff, fg_mask)

    # Find contours to track the objects
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around detected objects
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # filter small contours to remove noise
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(current_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Object Tracking', current_frame)

    # Prepare for next iteration
    prev_frame = current_frame.copy()
    ret, current_frame = cap.read()

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


: 